In [202]:
# load stock api
# alpaca trade api
# Initial imports
import os
import requests
##Imports

import pandas as pd
import numpy as np
#from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
%matplotlib inline

import yfinance as yf
from datetime import date
from yahoo_fin.stock_info import get_data
import datetime
import time
from yahoo_fin import stock_info as si
## MCForecastTools for future machine learning to predict
from MCForecastTools import MCSimulation
import json
import sys

In [203]:
# load goog news api
# # https://github.com/ranahaani/GNews
# # the reference also should consider google's own https://developers.google.com/webmaster-tools/v1/searchanalytics?hl=zh-cn


## to be simpler, use google trends api
## https://www.youtube.com/watch?v=WSnZrEI-sZ0
## https://pypi.org/project/pytrends/

from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=300) ## US/Eastern time zone


# Part 1: Get SPY and TSLA closing prices and daily returns in recnt five years

In [204]:
## load SPY and TSLA to track their stock performances and trends
## clear data
# load the tickers lists for three ETFs sectors
yf.pdr_override() 

# Create the Alpaca API object
SPY = yf.Ticker("SPY")#info
TSLA = yf.Ticker("TSLA")

In [205]:
#Get five years worth of historical data for ticker_list
SPY_df = SPY.history(period = "5y", interval = "1d")
SPY_df.index.duplicated().sum()
SPY_df = SPY_df.loc[~SPY_df.index.duplicated(keep = "first")]
SPY_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2018-04-26 00:00:00-04:00,243.158141,245.417165,242.698988,244.553955,67731900,0.0,0.0,0.0
2018-04-27 00:00:00-04:00,245.187641,245.499862,243.810183,244.783585,57053600,0.0,0.0,0.0
2018-04-30 00:00:00-04:00,245.426372,246.004909,242.827551,242.901031,82182300,0.0,0.0,0.0
2018-05-01 00:00:00-04:00,242.313293,243.442819,240.697066,243.332626,74203400,0.0,0.0,0.0
2018-05-02 00:00:00-04:00,243.130614,243.975439,241.294002,241.698059,86368900,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2023-04-19 00:00:00-04:00,412.220001,415.079987,412.160004,414.140015,55227300,0.0,0.0,0.0
2023-04-20 00:00:00-04:00,411.209991,413.700012,410.269989,411.880005,75840400,0.0,0.0,0.0
2023-04-21 00:00:00-04:00,412.190002,412.679993,410.170013,412.200012,73436100,0.0,0.0,0.0


In [206]:
## the TSLA df
TSLA_df = TSLA.history(period = "5y", interval = "1d")
TSLA_df.index.duplicated().sum()
TSLA_df = TSLA_df.loc[~TSLA_df.index.duplicated(keep = "first")]
TSLA_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-04-26 00:00:00-04:00,18.583332,19.052668,18.433332,19.032000,65340000,0.0,0.0
2018-04-27 00:00:00-04:00,19.024668,19.631332,18.922001,19.605333,65469000,0.0,0.0
2018-04-30 00:00:00-04:00,19.573999,19.915333,19.500000,19.593332,63423000,0.0,0.0
2018-05-01 00:00:00-04:00,19.567333,20.054667,19.548000,19.994667,69384000,0.0,0.0
2018-05-02 00:00:00-04:00,19.904667,20.456667,19.851999,20.076668,134556000,0.0,0.0
...,...,...,...,...,...,...,...
2023-04-19 00:00:00-04:00,179.100006,183.500000,177.649994,180.589996,125732700,0.0,0.0
2023-04-20 00:00:00-04:00,166.169998,169.699997,160.559998,162.990005,210970800,0.0,0.0
2023-04-21 00:00:00-04:00,164.800003,166.000000,161.320007,165.080002,123352300,0.0,0.0


In [207]:
# # combined two databases
# combined_df = pd.concat([TSLA_df,SPY_df], axis = 1)
# combined_df

In [208]:
# Pick TSLA and SPY close prices
tsla_close_price = TSLA_df['Close']
spy_close_price = SPY_df['Close'] 

# Print TSLA and SPY close prices
print(f"Current TSLA closing price: ${tsla_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

Current TSLA closing price: $Date
2018-04-26 00:00:00-04:00     19.032000
2018-04-27 00:00:00-04:00     19.605333
2018-04-30 00:00:00-04:00     19.593332
2018-05-01 00:00:00-04:00     19.994667
2018-05-02 00:00:00-04:00     20.076668
                                ...    
2023-04-19 00:00:00-04:00    180.589996
2023-04-20 00:00:00-04:00    162.990005
2023-04-21 00:00:00-04:00    165.080002
2023-04-24 00:00:00-04:00    162.550003
2023-04-25 00:00:00-04:00    160.669998
Name: Close, Length: 1258, dtype: float64
Current SPY closing price: $Date
2018-04-26 00:00:00-04:00    244.553955
2018-04-27 00:00:00-04:00    244.783585
2018-04-30 00:00:00-04:00    242.901031
2018-05-01 00:00:00-04:00    243.332626
2018-05-02 00:00:00-04:00    241.698059
                                ...    
2023-04-19 00:00:00-04:00    414.140015
2023-04-20 00:00:00-04:00    411.880005
2023-04-21 00:00:00-04:00    412.200012
2023-04-24 00:00:00-04:00    412.630005
2023-04-25 00:00:00-04:00    406.079987
Name: Close

In [209]:
#calculate daily returns of both TSLA and SPY
tsla_returns = tsla_close_price.pct_change()
spy_returns = spy_close_price.pct_change()

tsla_returns

spy_returns
#print both returns
print(f"Current TSLA daily return: ${tsla_returns}")
print(f"Current SPY daily return: ${spy_returns}")

Current TSLA daily return: $Date
2018-04-26 00:00:00-04:00         NaN
2018-04-27 00:00:00-04:00    0.030125
2018-04-30 00:00:00-04:00   -0.000612
2018-05-01 00:00:00-04:00    0.020483
2018-05-02 00:00:00-04:00    0.004101
                               ...   
2023-04-19 00:00:00-04:00   -0.020183
2023-04-20 00:00:00-04:00   -0.097458
2023-04-21 00:00:00-04:00    0.012823
2023-04-24 00:00:00-04:00   -0.015326
2023-04-25 00:00:00-04:00   -0.011566
Name: Close, Length: 1258, dtype: float64
Current SPY daily return: $Date
2018-04-26 00:00:00-04:00         NaN
2018-04-27 00:00:00-04:00    0.000939
2018-04-30 00:00:00-04:00   -0.007691
2018-05-01 00:00:00-04:00    0.001777
2018-05-02 00:00:00-04:00   -0.006717
                               ...   
2023-04-19 00:00:00-04:00   -0.000169
2023-04-20 00:00:00-04:00   -0.005457
2023-04-21 00:00:00-04:00    0.000777
2023-04-24 00:00:00-04:00    0.001043
2023-04-25 00:00:00-04:00   -0.015874
Name: Close, Length: 1258, dtype: float64


In [210]:
# combined both db closing and return
tsla_pro = pd.concat([tsla_close_price, tsla_returns], axis = 1)
spy_pro = pd.concat([spy_close_price, spy_returns], axis = 1)
#rename return
tsla_pro.columns = ['Close','Return']
tsla_pro
#rename return
spy_pro.columns = ['Close','Return']
spy_pro

print(tsla_pro.head())
print(spy_pro.head())

                               Close    Return
Date                                          
2018-04-26 00:00:00-04:00  19.032000       NaN
2018-04-27 00:00:00-04:00  19.605333  0.030125
2018-04-30 00:00:00-04:00  19.593332 -0.000612
2018-05-01 00:00:00-04:00  19.994667  0.020483
2018-05-02 00:00:00-04:00  20.076668  0.004101
                                Close    Return
Date                                           
2018-04-26 00:00:00-04:00  244.553955       NaN
2018-04-27 00:00:00-04:00  244.783585  0.000939
2018-04-30 00:00:00-04:00  242.901031 -0.007691
2018-05-01 00:00:00-04:00  243.332626  0.001777
2018-05-02 00:00:00-04:00  241.698059 -0.006717


In [211]:
# convert datetime to date in columns
# https://stephenallwright.com/pandas-convert-datetime-to-date/#:~:text=It's%20simple%20to%20convert%20a,the%20DateTime%20column%20in%20question.

tsla_pro.reset_index(inplace= True)
tsla_pro['Date'] = tsla_pro['Date'].dt.date
tsla_pro.columns = ['Date','Close','Return']
tsla_pro

,Date,Close,Return
0,2018-04-26,19.032000,NaN
1,2018-04-27,19.605333,0.030125
2,2018-04-30,19.593332,-0.000612
3,2018-05-01,19.994667,0.020483
4,2018-05-02,20.076668,0.004101
...,...,...,...
1253,2023-04-19,180.589996,-0.020183
1254,2023-04-20,162.990005,-0.097458
1255,2023-04-21,165.080002,0.012823
1256,2023-04-24,162.550003,-0.015326


In [212]:
# spy
spy_pro.reset_index(inplace= True)
spy_pro['Date'] = spy_pro['Date'].dt.date
spy_pro.columns = ['Date','Close','Return']
spy_pro

,Date,Close,Return
0,2018-04-26,244.553955,NaN
1,2018-04-27,244.783585,0.000939
2,2018-04-30,242.901031,-0.007691
3,2018-05-01,243.332626,0.001777
4,2018-05-02,241.698059,-0.006717
...,...,...,...
1253,2023-04-19,414.140015,-0.000169
1254,2023-04-20,411.880005,-0.005457
1255,2023-04-21,412.200012,0.000777
1256,2023-04-24,412.630005,0.001043


In [232]:
# show the plots
spy_pro.hvplot(x= "Date")

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [231]:
# tsla
tsla_pro.hvplot(x= "Date")

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

# Part 2: Checking TSLA and SPY in google trends in recent five years

In [255]:
## https://pypi.org/project/pytrends/#trending-searches

#tsla list trends
tsla_list = ["TESLA","TSLA"]
pytrends.build_payload(tsla_list, cat=0, timeframe='today 5-y', geo='US', gprop='news')
tsla_trend_df = pytrends.interest_over_time()
tsla_trend_df["TSLA TOTAL"] = tsla_trend_df["TSLA"]+tsla_trend_df["TESLA"]
tsla_trend_df = tsla_trend_df.drop(['TESLA','TSLA','isPartial'], axis=1)
tsla_trend_df.reset_index(inplace= True)
tsla_trend_df.columns = ['Date','TSLA News Trend Count']
tsla_trend_df
tsla_trend_df.hvplot(x= "Date")

:Curve   [Date]   (TSLA News Trend Count)

In [233]:
#spy list trends
spy_list = ["SPY"]
pytrends.build_payload(spy_list, cat=0, timeframe='today 5-y', geo='US', gprop='news')
spy_trend_df = pytrends.interest_over_time()
spy_trend_df = spy_trend_df.drop('isPartial', axis=1)
spy_trend_df.reset_index(inplace= True)
spy_trend_df.columns = ['Date','SPY News Trend Count']
spy_trend_df
spy_trend_df.hvplot(x = "Date")

:Curve   [Date]   (SPY News Trend Count)

# Part 3.1: combine databases and news trends for SPY

In [250]:
## combine spy
# from pytz import timezone

# # create a timezone-naive datetime object
# dt_naive = pd.to_datetime('2023-04-25 10:00:00')

# # localize the datetime object to a specific timezone
# dt_aware = dt_naive.tz_localize(timezone('US/Eastern'))

#spy_plus = pd.concat([spy_pro, spy_trend_df],ignore_index=True)




# import colorcet as cc
# from colorcet.plotting import swatch

# swatch('CET_L4')
# mag_cmap = cc.CET_L4[::-1]
# swatch("CET_L4_r", mag_cmap)

spy_plus = spy_pro.append(spy_trend_df, ignore_index=True)
spy_plus
#spy_map = spy_plus.hvplot.points(hover_cols = 'SPY',figsize=(20, 10), frame_height = 200, frame_width = 300)
#spy_map

# ##deploy double y axises to show the 
from bokeh.models import GlyphRenderer, LinearAxis, LinearScale, Range1d

x = np.random.randint(10, size=(10, 3))
x[:, 2] = np.cumsum(x[:, 2], axis=0)

spy_plus = pd.DataFrame(x, columns=["Date", "Close", "SPY News Trend Count"])

# # INITIALIZE plots
plot_1 = spy_plus.hvplot.line(x="Date", y="Close", yaxis="left")
plot_2 = spy_plus.hvplot.bar(x="Date", y="SPY News Trend Count", yaxis="right")


def overlay_hook(plot, element):
    # Adds right y-axis
    p = plot.handles["plot"]
    p.extra_y_scales = {"right": LinearScale()}
    p.extra_y_ranges = {"right": Range1d(0, 50)}
    p.add_layout(LinearAxis(y_range_name="right"), "right")
    #find the last line and set it to right
    lines = [p for p in p.renderers if isinstance(p, GlyphRenderer)]
    lines[-1].y_range_name = "right"


(plot_1.opts(ylim=(0, 10)) * plot_2.opts(color="k")).opts(hooks=[overlay_hook])

:Overlay
   .Curve.I :Curve   [Date]   (Close)
   .Bars.I  :Bars   [Date]   (SPY News Trend Count)

In [253]:
#Another way to show the combined SPY dataframe in plot
spy_map = spy_trend_df.hvplot(x = "Date") + spy_pro.hvplot(x= "Date")
spy_map

:Layout
   .Curve.I     :Curve   [Date]   (SPY News Trend Count)
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [Date]   (value)

# Analysis for SPY
# In the two graphs above, we can see that the relations between the SPY closing price and its news trend are not significantly relavant, for example, the up and down during 2018 till 2022, the waves
# in the news trend are equally spreaded, even during the pandamic in 2020 which not shown significantly changing. While in 01/23/2023, it has surprisingly big risen in news trend. However, the changes
# in the closing price were quite different, such as we could see there was a deep fallen in 2020 but 01/23/2023 was a normal day in market performance.

# Conclusion: For SPY, its last five years' stcok closing price performance and its news trend don't have linear relation.

# Part 3.2: combine databases and news trends for TSLA

In [254]:
## combine tsla
tsla_plus = tsla_pro.append(tsla_trend_df, ignore_index=True)
tsla_plus

,Date,Close,Return,TSLA TOTAL
0,2018-04-26,19.032000,NaN,NaN
1,2018-04-27,19.605333,0.030125,NaN
2,2018-04-30,19.593332,-0.000612,NaN
3,2018-05-01,19.994667,0.020483,NaN
4,2018-05-02,20.076668,0.004101,NaN
...,...,...,...,...
1513,2023-03-19 00:00:00,NaN,NaN,29.0
1514,2023-03-26 00:00:00,NaN,NaN,38.0
1515,2023-04-02 00:00:00,NaN,NaN,41.0
1516,2023-04-09 00:00:00,NaN,NaN,33.0


In [260]:
# ##deploy double y axises to show the 
from bokeh.models import GlyphRenderer, LinearAxis, LinearScale, Range1d

x = np.random.randint(10, size=(10, 3))
x[:, 2] = np.cumsum(x[:, 2], axis=0)

tsla_plus = pd.DataFrame(x, columns=["Date", "Close", "TSLA News Trend Count"])

# # INITIALIZE plots
plot_1 = tsla_plus.hvplot.line(x="Date", y="Close", yaxis="left")
plot_2 = tsla_plus.hvplot.bar(x="Date", y="TSLA News Trend Count", yaxis="right")


def overlay_hook(plot, element):
    # Adds right y-axis
    p = plot.handles["plot"]
    p.extra_y_scales = {"right": LinearScale()}
    p.extra_y_ranges = {"right": Range1d(0, 50)}
    p.add_layout(LinearAxis(y_range_name="right"), "right")
    #find the last line and set it to right
    lines = [p for p in p.renderers if isinstance(p, GlyphRenderer)]
    lines[-1].y_range_name = "right"


(plot_1.opts(ylim=(0, 10)) * plot_2.opts(color="k")).opts(hooks=[overlay_hook])

:Overlay
   .Curve.I :Curve   [Date]   (Close)
   .Bars.I  :Bars   [Date]   (TSLA News Trend Count)

In [258]:
#Another way to show the combined TSLA dataframe in plot
tsla_map = tsla_trend_df.hvplot(x = "Date") + tsla_pro.hvplot(x= "Date")
tsla_map

:Layout
   .Curve.I     :Curve   [Date]   (TSLA News Trend Count)
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [Date]   (value)

# Analysis for TSLA
# In the two graphs shown above, the first one which has two yaxis didn't show any relavant information, which could see they don't have a clear linear relations.
# the second graphs which the left one is the news trend, we can see compared with SPY, TSLA had more attentions overall and it had more clearer pikes. However, compared with its stock market performance,
# on the right graph which shows it was pretty silent before 2020, almost a flat line. After 2020 it has risen rocket high but then several moments it flopped as well, which made it unstableduring the recent two years.
# Conclusion: TSLA is always a news star during past five years but in stock market it has been more risky if you didn't buy it three years ago.